# ImJoy JupyterLab Extension Demo

This notebook demonstrate how to use [Vizarr viewer](https://github.com/hms-dbmi/vizarr) with the ImJoy JupyterLab Extension, please make sure you have installed `imjoy-jupyterlab-extension`. If you use https://jupyter.imjoy.io it will be installed automatically. You can find an ImJoy icon in the notebook toolbar if the extension is correctly installed.

# Prepare the ImJoy plugin for zarr visualization

In [2]:
import zarr
from imjoy import api
from imjoy_rpc import register_default_codecs

register_default_codecs()

class Plugin:
    def __init__(self, images, view_state=None):
        if not isinstance(images, list):
            images = [images]
        self.images = images
        self.view_state = view_state

    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(
            type="vizarr", src="https://hms-dbmi.github.io/vizarr"
        )
        if self.view_state:
            await viewer.set_view_state(self.view_state)
        for img in self.images:
            await viewer.add_image(img)


def run_vizarr(images, view_state=None):
    api.export(Plugin(images, view_state))


# Create an OME Zarr image

In [3]:
import zarr
import numpy as np
import os
from skimage import data
from skimage.transform import pyramid_gaussian

# Modified from https://github.com/ome/ome-zarr-py/blob/master/tests/create_test_data.py
def create_ome_zarr(zarr_directory, dtype="f4"):

    base = np.tile(data.astronaut(), (4, 4, 1))
    gaussian = list(pyramid_gaussian(base, downscale=2, max_layer=3, multichannel=True))

    pyramid = []
    # convert each level of pyramid into 5D image (t, c, z, y, x)
    for pixels in gaussian:
        red = pixels[:, :, 0]
        green = pixels[:, :, 1]
        blue = pixels[:, :, 2]
        # wrap to make 5D: (t, c, z, y, x)
        pixels = np.array([np.array([red]), np.array([green]), np.array([blue])])
        pixels = np.array([pixels]).astype(dtype)
        pyramid.append(pixels)

    store = zarr.DirectoryStore(zarr_directory)
    grp = zarr.group(store, overwrite=True)
    paths = []
    for path, dataset in enumerate(pyramid):
        grp.create_dataset(str(path), data=pyramid[path])
        paths.append({"path": str(path)})

    image_data = {
        "id": 1,
        "channels": [
            {
                "color": "FF0000",
                "window": {"start": 0, "end": 1},
                "label": "Red",
                "active": True,
            },
            {
                "color": "00FF00",
                "window": {"start": 0, "end": 1},
                "label": "Green",
                "active": True,
            },
            {
                "color": "0000FF",
                "window": {"start": 0, "end": 1},
                "label": "Blue",
                "active": True,
            },
        ],
        "rdefs": {
            "model": "color",
        },
    }

    multiscales = [
        {
            "version": "0.1",
            "datasets": paths,
        }
    ]
    grp.attrs["multiscales"] = multiscales
    grp.attrs["omero"] = image_data

# Save it to /tmp/astronaut.zarr
create_ome_zarr("/tmp/astronaut.zarr")

/lib/python3.9/site-packages/scipy/linalg/special_matrices.py:845: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if kind is 'lower':
/lib/python3.9/site-packages/scipy/linalg/special_matrices.py:847: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif kind is 'upper':
/lib/python3.9/site-packages/scipy/stats/_binned_statistic.py:469: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if(statistic is not 'count' and Vlen != Dlen):


# Load the OME Zarr image

In [4]:
import zarr

multiscale_astronaut = zarr.open("/tmp/astronaut.zarr", mode="r") # open the zarr created above in jupyter kernel

# Display the zarr image with vizarr

In [5]:
# Create Zarr 
astronaut_img = { "source": multiscale_astronaut, "name": "astronaut" }

# Run vizarr
run_vizarr(astronaut_img)

<IPython.core.display.Javascript object>